
1. FunctionTool: Convert any Python function into a tool that an agent can use. It automatically figures out how the function works.
2. QueryEngineTool: A tool that lets agents use query engines. Since agents are built on query engines, they can also use other agents as tools.
3. Toolspecs: Sets of tools created by the community, which often include tools for specific services like Gmail.
4. Utility Tools: Special tools that help handle large amounts of data from other tools.

In [2]:
import os
from dotenv import load_dotenv
from llama_index.llms.openrouter import OpenRouter

load_dotenv()

OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables. Please check your .env file.")

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
     print("Warning: GOOGLE_API_KEY not found. Google Embedding might fail if not using ADC.") # Or raise error

/home/mutaician/AIEngineerLearning/HuggingFaceAgentsCourse/llamaindex/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [1]:
# function tool - wrap any function and make it available to agent

from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Useful for getting weather for a given location."""
    print(f"Getting weather for {location}")
    return f"The weather in {location} is sunny"

tool = FunctionTool.from_defaults(
    get_weather,
    name='my_weather_tool',
    description='Useful for getting weather for a given location'
)

tool.call("Kericho")


Getting weather for Kericho


ToolOutput(content='The weather in Kericho is sunny', tool_name='my_weather_tool', raw_input={'args': ('Kericho',), 'kwargs': {}}, raw_output='The weather in Kericho is sunny', is_error=False)

In [ ]:
# Query Engine tool
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

embed_model = GoogleGenAIEmbedding(model_name="text-embedding-004", api_key=GOOGLE_API_KEY)

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(vector_store,embed_model)

llm = OpenRouter(
    api_key=OPENROUTER_API_KEY,
    max_tokens=256,
    model="openrouter/optimus-alpha"
)

query_engine = index.as_query_engine(llm)
tool = QueryEngineTool.from_defaults(query_engine=query_engine,name='some useful name', description="some useful description")

await tool.acall(
    "Responds about research on the impact of AI on the future of work and society"
)

ToolOutput(content='There is no information available regarding research on the impact of AI on the future of work and society. The expertise described is centered on 19th-century Irish politics and the Irish Home Rule movement.', tool_name='some useful name', raw_input={'input': 'Responds about research on the impact of AI on the future of work and society'}, raw_output=Response(response='There is no information available regarding research on the impact of AI on the future of work and society. The expertise described is centered on 19th-century Irish politics and the Irish Home Rule movement.', source_nodes=[NodeWithScore(node=TextNode(id_='f1946029-1f1c-4a8e-96fa-aafb6f6fd90d', embedding=None, metadata={'file_path': '/home/mutaician/AIEngineerLearning/HuggingFaceAgentsCourse/llamaindex/data/persona_1005.txt', 'file_name': 'persona_1005.txt', 'file_type': 'text/plain', 'file_size': 85, 'creation_date': '2025-04-12', 'last_modified_date': '2025-04-12'}, excluded_embed_metadata_keys=['

In [5]:
# Toolspecs - collection of tools that work together harmoniously
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()
tool_spec_list

In [6]:
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a query string and the maximum number\nof results requested by the user\n   Returns: List of relevant message objects up to the maximum number of results.\n\nArgs:\n    query[str]: The user's query\n    max_results (Optional[int]): The maximum number of search results\n    to return.\n"),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\nCreate and insert a draft email.\n   Print the returned draft's message and id.\n   Returns: Draft object, including draft id and message meta data.\n\nArgs:\n    to (Optional[str]): The email addresses to send the message to\n    subject (Optional[str]): The subject for the event\n    message (Optional[str]): The message for the event\n"),
